# Data Retrieval III (SQL)

In this notebook, we will work with the following:

1. SELECT statements.
2. Aggregation.
3. Window functions.
4. Joins.

In [1]:
import pandas as pd
import wrds

# SQL

SQL is **structured query language**, and it is a way that we can specify to a database management system ("DBMS") the form of data that we would like it to return to us.
This is another deep topic, but like the others, we can accomplish a lot for research with some well-chosen basics.

A DBMS generally stores data in tables, which are 2D datasets like the pandas dataframes or stats software datasets that we are accustomed to using.
These tables are related to each other using keys in one-to-one, one-to-many, and many-to-one relationships, hence the name "relational database."

SQL in research is most helpful in two particular cases:

1. Retrieving data from a data service that runs a DBMS for us (e.g., WRDS).
2. Creating and using a local database to help deal with big data that is more granular than we ultimately need.

We will focus below on the first case.

# SELECT statements

A `SELECT` statement tells the DBMS that we would like to select certain data from a table.
Its basic anatomy is quite simple:

```sql
SELECT *
  FROM comp.funda;
```

Above, `SELECT *` means that we want to select every column.
This is generally bad form, because, in practice, we rarely need all of the columns.
`FROM comp.funda` tells the DMBS that we want the `comp.funda` table, which is the Compustat Daily Updates - Fundamentals Annual.
When using WRDS, the database names are available at the top of the variable descriptions for a given table/query form.

The semicolon at the end signifies the end of the query.
Unlike Python, SQL does not use whitespace as syntax, though there are style [conventions](https://www.sqlstyle.guide).

In [2]:
# Connect to the database.
# db = wrds.Connection()

In [3]:
# The line below will use your connection to create a pgpass file 
# that stores your login credentials for future use.
# db.create_pgpass_file()

In [4]:
# Creating a function to make querying a bit easier to type.
def query_wrds(query, user):
    with wrds.Connection(wrds_username=user) as db:
        data = db.raw_sql(query)
    return data

In [5]:
_USER = 'jtkiley'

_Q01 = """
SELECT * 
  FROM comp.funda 
 LIMIT 10
"""

df01 = query_wrds(_Q01, _USER)
df01.head(10)

Loading library list...
Done


,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,prcc_f,prch_f,prcl_f,adjex_f,rank,au,auop,auopic,ceoso,cfoso
0,001000,1961-12-31,1961.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,None,None,None,3.341831,None,None,None,None,None,None
1,001000,1962-12-31,1962.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,None,None,None,3.341831,None,None,None,None,None,None
2,001000,1963-12-31,1963.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,None,None,None,3.244497,None,None,None,None,None,None
3,001000,1964-12-31,1964.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,None,None,None,3.089999,None,None,None,None,None,None
4,001000,1965-12-31,1965.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,None,None,None,3.089999,None,None,None,None,None,None
5,001000,1966-12-31,1966.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,None,None,None,3.089999,None,None,None,None,None,None
6,001000,1967-12-31,1967.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,None,None,None,3.089999,None,None,None,None,None,None
7,001000,1968-12-31,1968.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,None,None,None,3.000000,None,None,None,None,None,None
8,001001,1982-12-31,1982.0,INDL,C,D,SUMM_STD,AMFD.,000165100,A & M FOOD SERVICES INC,...,None,None,None,NaN,None,None,None,None,None,None
9,001000,1969-12-31,1969.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,None,None,None,1.000000,None,None,None,None,None,None


In [6]:
# We can look at all of these column names if we like.
# df01.columns.to_list()

Note two things in particular in the query and results above.

First, I used the `LIMIT` keyword with a value of `10`.
Compustat is a huge dataset, and retrieving everything would be a big download.
While we are experimenting or iterating on a query, using `LIMIT` asks the server to provide only a number of results up to the parameter to limit.
This is a strong norm when using this kind of data, as it dramatically reduces the load on the server.
`LIMIT` becomes more important as we ask the server to do transformation work for us, which increases the computational demand.

Second, there are 948 columns in this dataset.
Chances are, this is many more than we want, so we should narrow down to the variables of interest.

In [7]:
_Q02 = """
SELECT gvkey, fyear, conm, tic, cusip
       , at, lt
  FROM comp.funda
 WHERE (datafmt = 'STD')
 LIMIT 10
"""

df02 = query_wrds(_Q02, _USER)
df02.head(10)

Loading library list...
Done


,gvkey,fyear,conm,tic,cusip,at,lt
0,001000,1961.0,A & E PLASTIK PAK INC,AE.2,000032102,NaN,NaN
1,001000,1962.0,A & E PLASTIK PAK INC,AE.2,000032102,NaN,NaN
2,001000,1963.0,A & E PLASTIK PAK INC,AE.2,000032102,NaN,0.345
3,001000,1964.0,A & E PLASTIK PAK INC,AE.2,000032102,1.416,0.809
4,001000,1965.0,A & E PLASTIK PAK INC,AE.2,000032102,2.310,1.818
5,001000,1966.0,A & E PLASTIK PAK INC,AE.2,000032102,2.430,1.596
6,001000,1967.0,A & E PLASTIK PAK INC,AE.2,000032102,2.456,1.712
7,001000,1968.0,A & E PLASTIK PAK INC,AE.2,000032102,5.922,3.351
8,001000,1969.0,A & E PLASTIK PAK INC,AE.2,000032102,28.712,18.501
9,001000,1970.0,A & E PLASTIK PAK INC,AE.2,000032102,33.450,22.906


There are two changes above.
First, we picked explicit column names.

Second, we added a `WHERE` clause to impose a condition on the data that we want back.
In this case, we asked for rows where the column `datafmt` has a value of `STD`.
The default query form for Compustat returns only these standard data formats, so we recreate that here.

In [8]:
_Q03 = """
SELECT gvkey, fyear, conm, tic
       , cusip AS cusip9
       , SUBSTRING(cusip, 1, 8) AS cusip8
       , at, lt
  FROM comp.funda
 WHERE (datafmt = 'STD') AND 
       (fyear BETWEEN 2000 AND 2020)
 LIMIT 10
"""

df03 = query_wrds(_Q03, _USER)
df03.head(10)

Loading library list...
Done


,gvkey,fyear,conm,tic,cusip9,cusip8,at,lt
0,001004,2000.0,AAR CORP,AIR,000361105,00036110,701.854,361.642
1,001010,2000.0,ACF INDUSTRIES INC,4165A,00099V004,00099V00,3794.500,2794.500
2,001013,2000.0,ADC TELECOMMUNICATIONS INC,ADCT.1,000886309,00088630,3970.500,1057.800
3,001019,2000.0,AFA PROTECTIVE SYSTEMS INC,AFAP,001038108,00103810,28.638,15.454
4,001021,2000.0,AFP IMAGING CORP,IWKS,001058205,00105820,11.608,7.153
5,001034,2000.0,ALPHARMA INC -CL A,ALO.2,020813101,02081310,1610.435,762.548
6,001036,2000.0,UNITED DOMINION INDUSTRIES,UDI.,909914103,90991410,2325.377,1403.602
7,001037,2000.0,AM COMMUNICATIONS INC,AMCM.,001674100,00167410,6.373,4.253
8,001038,2000.0,AMC ENTERTAINMENT INC -OLD,AEN.2,001669100,00166910,1047.264,1106.309
9,001045,2000.0,AMERICAN AIRLINES GROUP INC,AAL,02376R102,02376R10,26213.000,19037.000


Here, we made three more changes.
First, we asked for the `cusip` column to be called `cusip9` in our results using `AS`.
Second, we used a function to transform the `cusip` column (using the `SUBSTRING()` function) to give us only eight characters and to name it `cusip8`.
This is a simple example of having the server do prep work for us.
Finally, we added a second condition to `WHERE`, a year restriction.

# Aggregation

Sometimes, the data in a table is more granular than the data that we want out.
So, we can ask the server to aggregate it for us, returning an aggregated dataset.

There are a few important things to know:

1. We use `GROUP BY` to tell the DBMS how to group rows before aggregating.
2. Every column must either be in the `GROUP BY` or have an aggregation function applied. A notable example here is that we ask for the `MAX` of the company name. If the name changes in the rows of the search, the DBMS would need to know how to choose. However, this is enforced as a general rule, not only when there is an actual conflict to resolve.
3. Order of the statements matter. For example, `WHERE` needs to be after `FROM` and before `GROUP BY`. I've done them here, so it will work, but this is a topic better explored in a book on the topic.

In [9]:
_Q10 = """
  SELECT gvkey
         , MAX(conm) AS co_name
         , AVG(at) AS assets_avg
         , SUM(ni) AS netincome_total
    FROM comp.funda
   WHERE (datafmt = 'STD') AND 
         (fyear BETWEEN 2000 AND 2020)
GROUP BY gvkey
   LIMIT 10
"""

df10 = query_wrds(_Q10, _USER)
df10.head(10)

Loading library list...
Done


,gvkey,co_name,assets_avg,netincome_total
0,001004,AAR CORP,1382.254550,665.734
1,001010,ACF INDUSTRIES INC,4013.050000,657.800
2,001013,ADC TELECOMMUNICATIONS INC,1817.245455,-1796.400
3,001019,AFA PROTECTIVE SYSTEMS INC,29.737000,3.052
4,001021,AFP IMAGING CORP,12.383111,-15.449
5,001034,ALPHARMA INC -CL A,1808.658000,-180.239
6,001036,UNITED DOMINION INDUSTRIES,2325.377000,56.550
7,001037,AM COMMUNICATIONS INC,12.120000,1.834
8,001038,AMC ENTERTAINMENT INC -OLD,1331.166500,-148.370
9,001045,AMERICAN AIRLINES GROUP INC,37019.571429,-6974.000


# Window functions

Sometimes, we want data at the level of the table, but we would also like aggregated measures.
SQL has something called **window fuctions** which aggregate data like we did before, but then they **broadcast** it up to the level of the original table.

In [10]:
_Q20 = """
  SELECT gvkey, fyear, conm, tic
         , cusip AS cusip9
         , SUBSTRING(cusip, 1, 8) AS cusip8
         , at, lt
         , AVG(at) OVER(PARTITION BY gvkey) AS assets_avg
         , SUM(ni) OVER(PARTITION BY gvkey) AS netincome_total
    FROM comp.funda
   WHERE (datafmt = 'STD') AND 
         (fyear BETWEEN 2000 AND 2020)
   LIMIT 10
"""

df20 = query_wrds(_Q20, _USER)
df20.head(10)

Loading library list...
Done


,gvkey,fyear,conm,tic,cusip9,cusip8,at,lt,assets_avg,netincome_total
0,001004,2000.0,AAR CORP,AIR,000361105,00036110,701.854,361.642,1382.25455,665.734
1,001004,2001.0,AAR CORP,AIR,000361105,00036110,710.199,399.964,1382.25455,665.734
2,001004,2002.0,AAR CORP,AIR,000361105,00036110,686.621,391.633,1382.25455,665.734
3,001004,2003.0,AAR CORP,AIR,000361105,00036110,709.292,407.608,1382.25455,665.734
4,001004,2004.0,AAR CORP,AIR,000361105,00036110,732.230,417.486,1382.25455,665.734
5,001004,2005.0,AAR CORP,AIR,000361105,00036110,978.819,556.102,1382.25455,665.734
6,001004,2006.0,AAR CORP,AIR,000361105,00036110,1067.633,573.390,1382.25455,665.734
7,001004,2007.0,AAR CORP,AIR,000361105,00036110,1362.010,776.755,1382.25455,665.734
8,001004,2008.0,AAR CORP,AIR,000361105,00036110,1377.511,720.616,1382.25455,665.734
9,001004,2009.0,AAR CORP,AIR,000361105,00036110,1501.042,754.692,1382.25455,665.734


Notice a few things about using window functions:

1. We're broadcasting back to the original row level, so there's no need to provide aggregation on the name.
2. We removed `GROUP BY`.
3. Instead, each aggregation function uses the `OVER()` function (which tells the DBMS that we want a window function), and, inside, it has `PARTITION BY` which serves the purpose of defining how the aggregation is done.

Window functions are very useful for a lot of the work we do, and they can easily push work to the server that we might otherwise have to do after retrieving the data.

# Joining data

A `JOIN` is combining one table with another (or multiple others) in order to query combined data.
This is a fairly deep topic, though we are going to work through a simple example.

In [11]:
_Q30 = """
  SELECT f.gvkey, f.fyear, f.conm, f.tic
         , f.cusip AS cusip9
         , SUBSTRING(f.cusip, 1, 8) AS cusip8
         , f.at, f.lt
         , AVG(f.at) OVER(PARTITION BY f.gvkey) AS assets_avg
         , SUM(f.ni) OVER(PARTITION BY f.gvkey) AS netincome_total
         , c.city
         , c.state
    FROM comp.funda AS f
    JOIN comp.company AS c
      ON f.gvkey = c.gvkey
   WHERE (f.datafmt = 'STD') AND 
         (f.fyear BETWEEN 2000 AND 2020)
   LIMIT 10
"""

df30 = query_wrds(_Q30, _USER)
df30.head(10)

Loading library list...
Done


,gvkey,fyear,conm,tic,cusip9,cusip8,at,lt,assets_avg,netincome_total,city,state
0,001004,2000.0,AAR CORP,AIR,000361105,00036110,701.854,361.642,1382.25455,665.734,Wood Dale,IL
1,001004,2001.0,AAR CORP,AIR,000361105,00036110,710.199,399.964,1382.25455,665.734,Wood Dale,IL
2,001004,2002.0,AAR CORP,AIR,000361105,00036110,686.621,391.633,1382.25455,665.734,Wood Dale,IL
3,001004,2003.0,AAR CORP,AIR,000361105,00036110,709.292,407.608,1382.25455,665.734,Wood Dale,IL
4,001004,2004.0,AAR CORP,AIR,000361105,00036110,732.230,417.486,1382.25455,665.734,Wood Dale,IL
5,001004,2005.0,AAR CORP,AIR,000361105,00036110,978.819,556.102,1382.25455,665.734,Wood Dale,IL
6,001004,2006.0,AAR CORP,AIR,000361105,00036110,1067.633,573.390,1382.25455,665.734,Wood Dale,IL
7,001004,2007.0,AAR CORP,AIR,000361105,00036110,1362.010,776.755,1382.25455,665.734,Wood Dale,IL
8,001004,2008.0,AAR CORP,AIR,000361105,00036110,1377.511,720.616,1382.25455,665.734,Wood Dale,IL
9,001004,2009.0,AAR CORP,AIR,000361105,00036110,1501.042,754.692,1382.25455,665.734,Wood Dale,IL


There are a number of changes here to make the `JOIN` work.

1. Notice that we added prefixes to all of the variables in original tables. Without these qualifiers, those column names are ambiguous.
1. Otherwise, most things look similar until the `JOIN`.
1. The `JOIN` itself has two parts: the `JOIN` specifying the other table we want to join, and the `ON` specifying how to join (or merge) the two. In this case, we are using `f.gvkey` and `c.gvkey`.
1. Like we have before, we're using `AS` again, this time to give short names to the tables (to make those prefixes easier to type).

Joins are powerful, and they can allow us to push a lot of our prep work onto the server. In addition, with copyrighted data like this, sharing a query with someone else is a way of transmitting exactly (or close to) what you pulled, while letting them rely on their own licensed access to the data.

# Breakout Exercises (time permitting)

If time permits, do the following exercise.

## EX1: customize a query

Choose one of the queries above, and edit to make two changes:

1. Restrict the results to Apple and Microsoft, two firms we've used as examples before. (Hint: the ticker symbols may be helpful)
2. Add an additional item of your choice to retrieve an additional column or aggregated variable.

In [ ]:
# 1-1
